<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/medcat_normalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install medcat
!python -m spacy download en_core_web_md
from medcat.cat import CAT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 48.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Load the model pack

medcat_zipfile_location = "/content/drive/My Drive/Colab Notebooks/Dissertation/MedCAT"

os.makedirs(medcat_zipfile_location, exist_ok=True)

medcat_model_zipfile = 'v2_SNOMED_2025_MIMIC_IV_bbe806e192df009f.zip'

MEDCAT_LOAD = os.path.join(medcat_zipfile_location, medcat_model_zipfile)

cat = CAT.load_model_pack(MEDCAT_LOAD)

In [ ]:
# Test

text = "The patient has a history of ectopic pregnancy but no current symptoms of miscarriage."
entities = cat.get_entities(text)

# This returns a dictionary of entities found

for entity_id, entity_data in entities['entities'].items():
    print(f"Term: {entity_data['source_value']}")
    print(f"SNOMED/CUI: {entity_data['cui']}")
    print(f"Confidence: {entity_data['context_similarity']}")

    # If using meta-annotations (Status/Certainty):
    print(f"Meta-Annotations: {entity_data.get('meta_anns')}")